In [1]:
import os
import json
import logging
import time
import copy

from src.log import setup_logger, get_logger
import config
from agents import agent_rewrite_question, agent_extract_company
from workflows import sql_query, check_db_structure
from src.utils import show
from utils import get_constant_column_list, ajust_org_question
from main import process_team_question, process_single_question

os.environ['DEBUG'] = '0'
os.environ['SHOW_LLM_INPUT_MSG'] = '1'

TableGraph: 已从 /data/workspace/howard/jinglever/competition/2024-FinGLM2-semi-final/cache/table_relations.json 加载表关系图


In [2]:
# 计算tokens数
total_usage_tokens = {
    agent_extract_company.cfg.name: 0,
    agent_rewrite_question.cfg.name: 0,
    check_db_structure.name: 0,
    sql_query.name: 0,
}

for q_team in config.all_question:
    if "usage_tokens" in q_team:
        for key in q_team["usage_tokens"]:
            if key in total_usage_tokens:
                total_usage_tokens[key] += q_team["usage_tokens"][key]

print(json.dumps(total_usage_tokens, ensure_ascii=False, indent=4))

total_tokens = sum(total_usage_tokens.values())
print(f"所有tokens数: {total_tokens}")

{
    "extract_company": 0,
    "rewrite_question": 0,
    "check_db_structure": 0,
    "Sql_query": 0
}
所有tokens数: 0


## 跑指定问题

In [3]:
t_idx = 0 # team index
q_idx = 0 # question index in team
config.FLAG_IGNORE_CACHE = True
question_team = config.all_question[t_idx]
if config.ENABLE_BATCH_MODE:
    process_team_question(question_team, t_idx)
    show(question_team["team"])
else:
    process_single_question(question_team, q_idx)
    show(question_team["team"][q_idx])
with open(config.OUTPUT_DIR+'/all_question.json', 'w', encoding='utf-8') as f:
    json.dump(config.all_question, f, ensure_ascii=False, indent=4)  # 添加缩进以便于阅读

result_commit = copy.deepcopy(config.all_question)
for q_team in result_commit:
    if "usage_tokens" in q_team:
        del q_team["usage_tokens"]
    if "use_time" in q_team:
        del q_team["use_time"]
    if "facts" in q_team:
        del q_team["facts"]
    if "sql_results" in q_team:
        del q_team["sql_results"]
    if "done" in q_team:
        del q_team["done"]
    for q_item in q_team["team"]:
        if "usage_tokens" in q_item:
            del q_item["usage_tokens"]
        if "use_time" in q_item:
            del q_item["use_time"]
        if "iterate_num" in q_item:
            del q_item["iterate_num"]
        if "facts" in q_item:
            del q_item["facts"]
        if "rewrited_question" in q_item:
            del q_item["rewrited_question"]
        if "sql_results" in q_item:
            del q_item["sql_results"]

with open(config.OUTPUT_DIR+'/Eva_Now_result.json', 'w', encoding='utf-8') as f:
    json.dump(result_commit, f, ensure_ascii=False, indent=4)  # 添加缩进以便于阅读

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.563 seconds.
Prefix dict has been built successfully.


>>>>> id: tttt----1----28-1-1
>>>>> Original Question: 天士力在2020年的最大担保金额是多少？答案需要包含1位小数
>>>>> Rewrited Question: 天士力在2020年的最大担保金额是多少？答案需要包含1位小数
>>>>> Answer: 1620000000.0元
>>>>> Used Time: 1m 5s

{
  "id": "tttt----1----28-1-1",
  "question": "天士力在2020年的最大担保金额是多少？答案需要包含1位小数",
  "answer": "1620000000.0元",
  "usage_tokens": {
    "extract_company": 142,
    "rewrite_question": 0,
    "check_db_structure": 14438,
    "Sql_query": 12116
  },
  "use_time": "1m 5s",
  "facts": [
    "天士力的关联信息有:[所在数据表是constantdb.secumain;InnerCode(证券内部编码)是1700;CompanyCode(公司代码)是1474;ChiName(中文名称)是天士力医药集团股份有限公司;EngName(英文名称)是Tasly Pharmaceutical Group Co.,Ltd.;SecuCode(证券代码)是600535;ChiNameAbbr(中文名称缩写)是天士力;EngNameAbbr(英文名称缩写)是TASLY;SecuAbbr(证券简称)是天士力;ChiSpelling(拼音证券简称)是TSL;]"
  ],
  "rewrited_question": "天士力在2020年的最大担保金额是多少？答案需要包含1位小数",
  "sql_results": [
    "查询sql```SELECT MAX(LatestGuaranteeSum) AS MaxGuaranteeAmount FROM astockeventsdb.lc_warrant AS w WHERE w.CompanyCode = 1474 AND DATE(w.InitialInfoPublDate